In [1]:
# Set up the session for V-Order writing
"spark.sql.parquet.vorder.enabled", "true"
"spark.microsoft.delta.optimizeWrite.enabled", "true"
"spark.microsoft.delta.optimizeWrite.binSize", "1073741824"

StatementMeta(, 2e558a9a-1180-4127-abee-4a0dd142a8fd, 3, Finished, Available, Finished)

('spark.microsoft.delta.optimizeWrite.binSize', '1073741824')

In [2]:
from pyspark.sql.functions import *

# Load data to the DataFrame
dim_date = spark.read.table("silver.adventureworks.hist_salesorderheader") \
.where(col("current") == True)

dim_date = dim_date.dropDuplicates(["OrderDate"]).select(col("OrderDate"), \
        dayofmonth("OrderDate").alias("Day"), \
        month("OrderDate").alias("Month"), \
        year("OrderDate").alias("Year")
    ).orderBy("OrderDate")

# Add hash code using all selected columns
dim_date = dim_date.withColumn("ID", \
sha2(concat_ws("||", *dim_date.columns), 256))

StatementMeta(, 2e558a9a-1180-4127-abee-4a0dd142a8fd, 4, Finished, Available, Finished)

In [3]:
from delta.tables import *

deltaTable = DeltaTable.forPath(spark, \
'Tables/adventureworks/dimension_date')
       
deltaTable.alias('silver') \
  .merge(
    dim_date.alias('updates'),
    'silver.ID = updates.ID'
  ).whenNotMatchedInsert(values =
    {
      "ID": "updates.ID",
      "OrderDate": "updates.OrderDate",
      "Day": "updates.Day",
      "Month": "updates.Month",
      "Year": "updates.Year",
    }
  ).execute()

StatementMeta(, 2e558a9a-1180-4127-abee-4a0dd142a8fd, 5, Finished, Available, Finished)